The investigative reporter team from Buzzfeed published source material on organizations and people related to Donald Trump.

<!-- todo embed tweet? -->

The group asked the public for help analysing, extending and reporting on the material.

The data was published in this [google doc](https://docs.google.com/spreadsheets/d/1Z5Vo5pbvxKJ5XpfALZXvCzW26Cl4we3OaN73K9Ae5Ss/edit#gid=634968401).

Each tab contained information on a pair of organizations or people and the type of connection as well as the source.

<!-- todo embed gdoc? -->

## Accessing the Data

With the Neo4j Cypher query language, we can use the `LOAD CSV` clause to access tabular data to render, compute or integrate in the graph. As the google doc is publicly readable, we can download each tab as CSV or TSV and grab the **download URL** which we will use.

In this notebook we will use the official `neo4j-python-driver` to talk to our local Neo4j instance.

In [12]:
# !pip install neo4j-driver

import os
from neo4j.v1 import GraphDatabase, basic_auth

from scripts.vis import draw_query
options = {"Person": "name","Bank": "name", "Organization": "name"}

neo4jUrl = os.environ.get('NEO4J_URL',"bolt://localhost")
neo4jUser = os.environ.get('NEO4J_USER',"neo4j")
neo4jPass = os.environ.get('NEO4J_PASSWORD',"test")

driver = GraphDatabase.driver(neo4jUrl, auth=basic_auth(neo4jUser, neo4jPass))

session = driver.session()

In [ ]:
url = 'https://docs.google.com/spreadsheets/u/1/d/1Z5Vo5pbvxKJ5XpfALZXvCzW26Cl4we3OaN73K9Ae5Ss/export?format=tsv&id=1Z5Vo5pbvxKJ5XpfALZXvCzW26Cl4we3OaN73K9Ae5Ss'
org_org_url = url + '&gid=634968401'
person_org_url = url + '&gid=1368567920'
person_person_url = url + '&gid=905294723'
LOAD_CSV = "LOAD CSV WITH HEADERS FROM {url} AS row FIELDTERMINATOR '\t'"

result = session.run(LOAD_CSV + " RETURN row limit 5", url=org_org_url)
for record in result:
    print(record)

In [ ]:
import_org_org = LOAD_CSV + """
MERGE (o1:Organization {name:row.`Organization A`})
MERGE (o2:Organization {name:row.`Organization B`})
CREATE (o1)-[con:CONNECTED_TO]->(o2)
SET con.connection=row.Connection, con.source=row.`Source(s)`
"""
print(session.run(import_org_org, url=org_org_url).consume().counters)

In [ ]:
import_person_org = LOAD_CSV + """
MERGE (p:Person {name:row.Person})
MERGE (o:Organization {name:row.Organization})
CREATE (p)-[con:INVOLVED_WITH]->(o)
SET con.connection=row.Connection, con.source=row.`Source(s)`
"""
print(session.run(import_person_org, url=person_org_url).consume().counters)

In [ ]:
import_person_person = LOAD_CSV + """
MERGE (p1:Person {name:row.`Person A`})
MERGE (p2:Person {name:row.`Person B`})
CREATE (p2)-[con:RELATED_TO]->(p1)
SET con.connection=row.Connection, con.source=row.`Source(s)`
"""
print(session.run(import_person_person, url=person_person_url).consume().counters)

In [ ]:
stats = """
MATCH (o:Organization)-[r]-()
RETURN o.name as name,type(r) as type, count(*) as degree
"""
session.run(stats)

In [13]:
banks = """
MATCH (bank:Organization)-[r]-(other) 
WHERE bank.name CONTAINS 'BANK' 
RETURN bank,r,other
"""
draw_query(session, banks, options)

In [8]:
draw_query(session, "MATCH (p:Person {name:'REX TILLERSON'})-[r]-(other) RETURN p,r,other LIMIT 25", options)